In [ ]:
import warnings
warnings.filterwarnings("ignore")

In this Notebook we will go over what K-Means Clustering is, and how to implement it with RAPIDS cuML.

This Notebook can be run with a free GPU at [app.blazingsql.com](https://bit.ly/intro_ds_notebooks): `git clone https://github.com/Dropout-Analytics/cuml_kmeans`

# [Intro to K-Means Clustering with cuML](https://medium.com/dropout-analytics/intro-to-k-means-clustering-with-cuml-b6d617e36456?source=friends_link&sk=ce3b63fd8f41c9bdfbd04fa4f40b2365)

K-Means is an easy way to cluster data. It randomly selects K points (centroids) in a given dataset, then computes which of the dataset's instances are closest to each point (making a cluster). 

For every cluster, the mean of its values (instances) is computed, and this mean becomes that cluster's new point (centroid).

Once a cluster's centroid has moved, its distance from the dataset's instances has changed, and instances may be added to or removed from that cluster. The mean is recalculated & replaced until it stops moving or has hit the given number maximum iterations (`max_iter`), whichever comes first.

![K-Means visual](https://cdn-images-1.medium.com/max/800/0*AVBuF9liGiN3RaR6.gif)

For this intro, we'll be working with [Fischer's Iris dataset](https://wikipedia.org/wiki/Iris_flower_data_set). The set holds 3 species of Iris flower (Iris setosa, Iris virginica and Iris versicolor).

In [ ]:
import cudf

df = cudf.read_csv('https://github.com/gumdropsteve/datasets/raw/master/iris.csv')

![Iris species](https://camo.githubusercontent.com/224f964448da6133df6dcb316c6a0352a5403de6/68747470733a2f2f6d69726f2e6d656469756d2e636f6d2f6d61782f3630302f302a585762417a384a53704478736e633864)

In [ ]:
df.species.unique()

4 features - the length & width of the sepals & petals -  were measured in centimeters from each flower; each species was recorded 50 times.

In [ ]:
df.tail(3)

#### Visualize 
Let's see what the model is working with. As we already know this dataset's target values (`species`), we can easily visualize the different clusters with Matplotlib. Our K-Means model will not be given this information.

In [ ]:
df.to_pandas().plot(kind='scatter', x='petal_width', y='sepal_length', 
                    c='target', cmap=('rainbow'), sharex=False)

## cuML KMeans
> cuML’s [KMeans](https://docs.rapids.ai/api/cuml/stable/api.html#k-means-clustering) supports the scalable [KMeans++](https://en.wikipedia.org/wiki/K-means%2B%2B) initialization method [(`init='scalable-k-means++'` or `'k-means||'`)]. This method is more stable than randomly selecting K points [(`init='random'`)]. 
>
> The model can take array-like objects, either in host as NumPy arrays or in device (as Numba or cuda_array_interface-compliant), as well as cuDF DataFrames as the input.

In [ ]:
from cuml.cluster import KMeans as KMeans

kmeans = KMeans(n_clusters=3, max_iter=300, init='scalable-k-means++')

As the purpose of K-Means is to determine the data's clusters, we don't need to give the model the `species` or `target` columns.

In [ ]:
X = df.drop(['species', 'target'])

`.fit()` the model with our training data. 

In [ ]:
kmeans.fit(X)

### Score Results
We can score our model with cuML's `adjusted_rand_score()` function, which is a [Rand index](https://en.wikipedia.org/wiki/Rand_index) adjusted for chance.

The Rand index (RI) computes a similarity measure between two clusterings by considering all pairs of samples and counting pairs that are assigned in the same or different clusters in the predicted and true clusterings. The raw RI score is then “adjusted for chance” into the Adjusted Rand index (ARI) score using the following scheme:
```python
ARI = (RI - Expected_RI) / (max(RI) - Expected_RI)
```

> The adjusted Rand index is thus ensured to have a value close to 0.0 for random labeling independently of the number of clusters and samples, exactly 1.0 when the clusterings are identical, and -1 (up to a permutation).

In [ ]:
from cuml.metrics import adjusted_rand_score

score = adjusted_rand_score(labels_true=df['target'], 
                            labels_pred=kmeans.labels_)

In [ ]:
score

#### Visualize Clusters

Let's add our `kmeans.labels_` to a `.copy()` of the original `df` and see how they look.

Note: The model was never made aware of the actual clusters, and came up with its own, so comparing the `target` and `predicted` patterns will give you a more accurate understanding than comparing the `predicted` and `actual` values here. _For high scores on this dataset, this usually means the 2s match up and the 1s and 0s are flipped._

In [ ]:
results_df = df.copy()

results_df['predicted'] = kmeans.labels_

results_df

With the difference in species' respective target values (so the colors will be in a different order) in mind, let's visualize our model's clusters.

In [ ]:
X.to_pandas().plot(kind='scatter', x='petal_width', y='sepal_length', 
                    c=kmeans.labels_.to_array(), cmap=('rainbow'), sharex=False)

# How to Determine K?
Well, it really helps if you just know how many clusters there are. But if you don't, there are a number of ways to figure out the best value for K.

### The 'Elbow' method
With the Elbow method, we simply plot the within-cluster sum of squares (RSS) and try to see what looks like an elbow.

In [ ]:
def kmean_score(nclust):
    km = KMeans(n_clusters=nclust)
    km.fit(X)
    rss = -km.score(X)
    return rss

In [ ]:
scores = [kmean_score(i) for i in range(1, 8)]

Picking the 'elbow' point allows us to capitalize on reduction in variation as K increases. You should pick K where the reduction in variation flattens off (comparatively). In our case, 3.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(range(1, 8), scores)
plt.xlabel('K')
plt.ylabel('RSS')
plt.title('RSS versus K')

### Silhouette score
The silhouette plot displays a measure of how close each point in one cluster is to points in the neighboring clusters and thus provides a way to assess parameters like number of clusters visually. 

For each point $x_i$:

- $a(i)$ average dissimilarity of $x_i$ with points in the same cluster
- $b(i)$ average dissimilarity of $x_i$ with points in the nearest cluster
    "nearest" means cluster with the smallest $b(i)$

$$\text{silhouette}(i) = \frac{b(i) - a(i)}{max(a(i), b(i))} $$

Silhouette scores range from -1 to 1. 

> Silhouette coefficients (as these values are referred to as) near +1 indicate that the sample is far away from the neighboring clusters. A value of 0 indicates that the sample is on or very close to the decision boundary between two neighboring clusters and negative values indicate that those samples might have been assigned to the wrong cluster.

The silhouette score of a clustering is the average of silhouette score of all points.

In [ ]:
from sklearn.metrics import silhouette_score

def get_silhouette_score(nclust):
    km = KMeans(n_clusters=nclust)
    km.fit(X)
    sil_avg = silhouette_score(X.to_pandas(), km.labels_.to_pandas())
    return sil_avg

In [ ]:
sil_scores = [get_silhouette_score(i) for i in range(2, 8)]

In [ ]:
plt.plot(range(2, 8), sil_scores)
plt.xlabel('K')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs K')

# Continued Learning 
Here are some resources I recommend to help fill in any gaps and provide a more complete picture.

#### **StatQuest: K-means clustering**
- Watch on YouTube: [https://youtu.be/4b5d3muPQmA](https://youtu.be/4b5d3muPQmA)
- Channel: StatQuest with Josh Starmer ([Subscribe](https://www.youtube.com/channel/UCtYLUTtgS3k1Fg4y5tAhLbw?sub_confirmation=1))

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('4b5d3muPQmA', width=(1280*0.69), height=(720*0.69))

#### k-means++:  The Advantages of Careful Seeding
- Stanford InfoLab Publication Server: [ilpubs.stanford.edu:8090/778/1/2006-13.pdf](http://ilpubs.stanford.edu:8090/778/1/2006-13.pdf)
- by David Arthur and Sergei Vassilvitskii

#### Scalable K-Means++
- Very Large Data Bases Endowment Inc.: [vldb.org/pvldb/vol5/p622_bahmanbahmani_vldb2012.pdf](http://vldb.org/pvldb/vol5/p622_bahmanbahmani_vldb2012.pdf)
- by Bahman Bahmani, Benjamin Moseley, Andrea Vattani, Ravi Kumar and Sergei Vassilvitskii

#### **K-means clustering**
Wikipedia: [wikipedia.org/wiki/K-means_clustering](https://wikipedia.org/wiki/K-means_clustering)

#### [**Back to GitHub**](https://github.com/Dropout-Analytics/cuml_kmeans) | [Sponsor this Project](https://github.com/sponsors/gumdropsteve)